## __**Capstone Project - The Battle of Neighborhoods (Week 1)**__ 

### Author - Vishal Kaistha

### __**1. A description of the problem and a discussion of the background.**__

### __1.1 Problem Statement__

#### Which neighborhood in Toronto should be picked for one of my clients for purchasing a new house or condo?

### __1.2 Background Discussion__

#### Choosing a location when looking for a home is very important! After all, you can always update or fix your house, but you can't easily change its location and the vibe of the community! In this project, I would like to create a hypothetical business scenario with an aim of finding a compatible location for one of my clients.

### __1.3 Who is my audience and why would they care?__

#### Although many zones already have their own labels, such as 'China Town', 'Little Italy', however, the overall perceived quality of a community is highly subjective to different clients, because they will have their own needs and expectation. Also, in this project, the 'hypothetical client' has never lived in Toronto before, so it will be very convenient for him if I could provide a smart 'recommendation system' for him based on his own expectation and rating of various aspects, the data retrieved from FourSquare as well as a recommendation engine designed and customized for him.

## __**2. A description of the data and how it will be used to solve the problem.**__

#### In this project, I will use raw data provided by wikipedia, geographical coordinates,data retrieved from FourSquare and a dataset provided by the customer as his own expectation/rating.

#### 2.1 Data from Wikipedia contains a list of postal codes of Canada. It will be retreived by scraping a table from the website. This data is used to create a geographical segmentation of Toronto based on postal code, and it will be linked with the actual coordinates later. Note that the methodology used in this subsection overlaps with the previous project, so feel free to skip through it if you are already familiar with it.

In [1]:
!pip install bs4
!pip install lxml

  Stored in directory: /home/dsxuser/.cache/pip/wheels/a0/b0/b2/4f80b9456b87abedbc0bf2d52235414c3467d8889be38dd472
Successfully built bs4


In [2]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from urllib.request import urlopen
import urllib.request
!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    certifi-2020.4.5.1         |   py36h9f0ad1d_0         151 KB  conda-forge
    altair-4.1.0               |             py_1         614 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    ca-certificates-2020.4.5.1 |       hecc5488_0         146 KB  conda-forge
    branca-0.4.0               |             py_0          26 KB  conda-forge
    openssl-1.1.1f             |       h516909a_0         2.1 MB  conda-forge
    ------------------------------------------------------------
                       

#### Obtain wikipedia table and perform data wrangling

In [3]:
#Obtain the Wikipedia article as a local copy.
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
request = urllib.request.urlopen(url)
wiki_article = request.read().decode()

with open('List_of_postal_codes_of_Canada:_M.html', 'w') as fo:
    fo.write(wiki_article)
    

# Load article, use beautiful soup to get the tables.
wiki_article = open('List_of_postal_codes_of_Canada:_M.html').read()
soup = BeautifulSoup(wiki_article, 'html.parser')
tables = soup.find_all('table', class_='sortable')

# Search through all the tables, identify the table with the header we want.
for table in tables:
    all_tables = table.find_all('th')
    header = [th.text.strip() for th in all_tables]
    if header[:5] == ['Postcode', 'Borough', 'Neighborhood']:
        break

# Extract the columns we want and write to a semicolon-delimited text file.
with open('List_of_postal_codes_of_Canada:_M.txt', 'w') as fo:
    for tr in table.find_all('tr'):
        tds = tr.find_all('td')
        if not tds:
            continue
        Postcode, Borough, Neighborhood = [td.text.strip() for td in tds[:4]]
        
        print('; '.join([Postcode, Borough, Neighborhood]), file=fo)
        
# Load to pandas dataframe        
df = pd.read_table('List_of_postal_codes_of_Canada:_M.txt', delimiter = ';', header = None)
df.columns = ['PostCode', 'Borough', 'Neighborhood']

# Ignore not assigned borough
df1 = df[df['Borough'] != ' Not assigned']
df1.reset_index(inplace = True, drop = True)

# Assign borough value to neighborhood if the neighborhood is not assigned.
position = 0
neigh_list = []
for i,j in zip(df1['Borough'], df1['Neighborhood']):
    if j == ' Not assigned':
        neigh_list.append(i)
    else:
        neigh_list.append(j)

post_list = df1['PostCode'].tolist()
br_list = df1['Borough'].tolist()

df1 = pd.DataFrame([post_list, br_list, neigh_list]).T
df1.columns = ['PostCode', 'Borough', 'Neighborhood']
df1.head()

# Group dataframe by PostCode and combine neighborhood values.
borough_list =[] 
Neighborhood_list = []

for item in df1['Borough']:
    item_new = str(item)[1:] + ':'
    borough_list.append(item_new)
    
for item in df1['Neighborhood']:
    item_new = str(item)[1:] + ':'
    Neighborhood_list.append(item_new)
    
PostCode_list = df1['PostCode'].tolist()

df2 = pd.DataFrame([PostCode_list, borough_list, Neighborhood_list]).T
df2.columns = ['PostCode', 'Borough', 'Neighborhood']

new_df = df2.groupby('PostCode').sum()

borough_list=[]
Neighborhood_list = []
PostCode_list = new_df.index.tolist()

for item in new_df['Borough']:
    item_new = np.unique(np.array(str(item).split(':')))[1]
    borough_list.append(item_new)

for item in new_df['Neighborhood']:
    item_new = str(np.array(str(item).split(':'))[:-1].tolist())[1:][:-1].replace("'","")
    Neighborhood_list.append(item_new)

df3 = pd.DataFrame([PostCode_list, borough_list, Neighborhood_list]).T
df3.columns = ['PostCode', 'Borough', 'Neighborhood']
df3

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:33: FutureWarning: read_table is deprecated, use read_csv instead.


,PostCode,Borough,Neighborhood
0,M1B,Scarborough,Malvern / Rouge
1,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek
2,M1E,Scarborough,Guildwood / Morningside / West Hill
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,Kennedy Park / Ionview / East Birchmount Park
7,M1L,Scarborough,Golden Mile / Clairlea / Oakridge
8,M1M,Scarborough,Cliffside / Cliffcrest / Scarborough Village West
9,M1N,Scarborough,Birch Cliff / Cliffside West


#### 2.2 Geographical coordinates. This dataset was provided by the previous project. It contains the coordinates for various postal code. The aim of this dataset is to link postal code with coordinates. Note that the methodology used in this subsection overlaps with the previous project, so feel free to skip through it if you are already familiar with it

In [4]:
# Load the csv file from source.
df_co = pd.read_csv('https://cocl.us/Geospatial_data')

lat_list = []
long_list = []

for PostCode_target in df3['PostCode']:
    for PostCode, Latitude, Longitude in zip (df_co['Postal Code'],
                                                 df_co['Latitude'],
                                               df_co['Longitude']):
        if PostCode_target == PostCode:
            lat_list.append(Latitude)
            long_list.append(Longitude)

# Add coordinates information to the pandas dataframe.
Final_df = pd.DataFrame([df3['PostCode'].tolist(), 
                         df3['Borough'].tolist(), 
                         df3['Neighborhood'].tolist(),
                         lat_list,
                         long_list]).T
Final_df.columns = ['PostCode','Borough','Neighborhood', 'Latitude', 'Longitude']
Final_df

,PostCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,Malvern / Rouge,43.8067,-79.1944
1,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek,43.7845,-79.1605
2,M1E,Scarborough,Guildwood / Morningside / West Hill,43.7636,-79.1887
3,M1G,Scarborough,Woburn,43.771,-79.2169
4,M1H,Scarborough,Cedarbrae,43.7731,-79.2395
5,M1J,Scarborough,Scarborough Village,43.7447,-79.2395
6,M1K,Scarborough,Kennedy Park / Ionview / East Birchmount Park,43.7279,-79.262
7,M1L,Scarborough,Golden Mile / Clairlea / Oakridge,43.7111,-79.2846
8,M1M,Scarborough,Cliffside / Cliffcrest / Scarborough Village West,43.7163,-79.2395
9,M1N,Scarborough,Birch Cliff / Cliffside West,43.6927,-79.2648


#### 2.3 Data retrieved from FourSquare will be used to evaluate each zone of interest. For each postal code area, obtain the frequency of occurrence of interesting venues by further data wrangling and preparation. Note that the methodology used in this subsection overlaps with the previous project, so feel free to skip through it if you are already familiar with it.

In [5]:
import requests # library to handle requests

import matplotlib.cm as cm
import matplotlib.colors as colors

import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [6]:
# Extract latitude and longitude of downtown Toronto
df_to = Final_df
lat = Final_df[Final_df['Borough']=='Downtown Toronto'].iloc[0].Latitude
long = Final_df[Final_df['Borough']=='Downtown Toronto'].iloc[0].Longitude

print('The geograpical coordinate of downtown Toronto City are {}, {}.'.format(lat, long))

# create map of Toronto using latitude and longitude values.
map_to = folium.Map(location=[lat, long], zoom_start=10)

# add markers to map
for lat, lng, borough, postcode in zip(df_to['Latitude'], df_to['Longitude'], df_to['Borough'], 
                                           df_to['PostCode']):
    label = '{}, {}'.format(postcode, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_to)  
    
map_to

The geograpical coordinate of downtown Toronto City are 43.6795626, -79.37752940000001.


In [33]:
import requests # library to handle requests

In [34]:
from pandas.io.json import json_normalize

# Enter the following information. The actual info was removed before sharing since it is sensitive. please see output.
CLIENT_ID = '4VWGB0NJSVQBJ5N0I44KZ5MJZLU1B0Z0IJ3AL3S1MJTC1AMP' # your Foursquare ID
CLIENT_SECRET = 'CENQHLGTNTHCPXP33YMCQGERRNQGPRT41WI3JKDFZ2VD2MTD'
VERSION = '20180605' # Foursquare API version

LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 300 # define radius

neighborhood_latitude = df_to[df_to['PostCode']=='M5C'].Latitude.iloc[0]
neighborhood_longitude = df_to[df_to['PostCode']=='M5C'].Longitude.iloc[0]

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)

# Retrieve information from FourSquare
# results = requests.get(url).json()

# This function is from applied data science course materials, which will be utilized by this assignment.
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
#         print(name)       
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']

        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])
    

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    
    nearby_venues.columns = ['PostCode','Neighborhood Latitude','Neighborhood Longitude','Venue','Venue Latitude','Venue Longitude','Venue Category']
    
    return(nearby_venues)

to_venues = getNearbyVenues(names=df_to['PostCode'], latitudes=df_to['Latitude'], longitudes=df_to['Longitude'])

In [35]:
# one hot encoding
to_onehot = pd.get_dummies(to_venues[['Venue Category']], prefix="", prefix_sep="")

# add postcal code column back to dataframe
to_onehot['PostCode'] = to_venues['PostCode'] 

# move postal code column to the first column
fixed_columns = [to_onehot.columns[-1]] + list(to_onehot.columns[:-1])
to_onehot = to_onehot[fixed_columns]

to_onehot.head()

,PostCode,Accessories Store,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,American Restaurant,Antique Shop,Aquarium,...,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,M1B,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,M1C,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,M1C,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,M1E,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,M1E,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [36]:
#Group rows by PostCode and take the mean of the frequency of occurrence of each category
to_grouped = to_onehot.groupby('PostCode').mean().reset_index()
to_grouped

,PostCode,Accessories Store,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,American Restaurant,Antique Shop,Aquarium,...,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,M1B,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,...,0.000,0.000000,0.000000,0.000000,0.000000,0.0,0.00000,0.0000,0.000000,0.000000
1,M1C,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,...,0.000,0.000000,0.000000,0.000000,0.000000,0.0,0.00000,0.0000,0.000000,0.000000
2,M1E,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,...,0.000,0.000000,0.000000,0.000000,0.000000,0.0,0.00000,0.0000,0.000000,0.000000
3,M1G,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,...,0.000,0.000000,0.000000,0.000000,0.000000,0.0,0.00000,0.0000,0.000000,0.000000
4,M1H,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,...,0.000,0.000000,0.000000,0.000000,0.000000,0.0,0.00000,0.0000,0.000000,0.000000
5,M1J,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,...,0.000,0.000000,0.000000,0.000000,0.000000,0.0,0.00000,0.0000,0.333333,0.000000
6,M1K,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,...,0.125,0.000000,0.000000,0.000000,0.000000,0.0,0.00000,0.0000,0.000000,0.000000
7,M1L,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,...,0.000,0.000000,0.000000,0.000000,0.000000,0.0,0.00000,0.0000,0.000000,0.000000
8,M1M,0.000000,0.000000,0.0,0.0,0.0,0.0,0.500000,0.000000,0.0,...,0.000,0.000000,0.000000,0.000000,0.000000,0.0,0.00000,0.0000,0.000000,0.000000
9,M1N,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,...,0.000,0.000000,0.000000,0.000000,0.000000,0.0,0.00000,0.0000,0.000000,0.000000


#### 2.4 Dataset provided by the customer as his own expectation/rating will be used to create a customized recommendation engine. Detailed data processing will be provided in the next project. Note that the metrics that are important to this customer might have broad and vague definition, so it will be carefully mapped to corresponding categories retrieved from FourSquare in the next project.

In [37]:
columns = ['Customer_Name', 'Cafe', 'Health Care', 'School', 'Gym', 'Full Score']
Rating = ['John', 7, 10, 9, 6, 10]
df_rating = pd.DataFrame([Rating])
df_rating.columns = columns
df_rating.set_index('Customer_Name')
df_rating

,Customer_Name,Cafe,Health Care,School,Gym,Full Score
0,John,7,10,9,6,10
